In [1]:
from model import Model
import torch
from transformers import AdamW,PretrainedConfig
from tqdm.auto import tqdm
from datasets import load_dataset
import numpy as np

In [2]:
ds_train = load_dataset("Salesforce/wikitext", "wikitext-103-v1",split="train",num_proc=8)
ds_validate = load_dataset("Salesforce/wikitext", "wikitext-103-v1",split="validation",num_proc=1)
ds_test = load_dataset("Salesforce/wikitext", "wikitext-103-v1",split="test",num_proc=1)

In [3]:
import torch.utils
import torch.utils.data


dl_train = torch.utils.data.DataLoader(ds_train,shuffle=False,batch_size=256)

In [4]:
model = Model(PretrainedConfig(max_length=256,min_length=256))

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to("cuda")

optimizer = AdamW(model.parameters(), lr=1e-5)

c:\Users\musasina\anaconda3\envs\mfp\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
num_epochs=100
for epoch in tqdm(range(num_epochs)):
    model.train()
    for step,batch in enumerate(dl_train):
        optimizer.zero_grad()
        x = np.array(batch["text"],dtype=np.str_)
        x[x==""] = " "
        tokeniezed = model.tokenizer(x.tolist(),return_tensors="pt",padding=True,truncation=True).to("cuda")
        print(tokeniezed["input_ids"])
        outputs = model(tokeniezed["input_ids"].float())
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        

  0%|          | 0/100 [00:00<?, ?it/s]

tensor([[  220, 50256, 50256,  ..., 50256, 50256, 50256],
        [  796,   569, 18354,  ..., 50256, 50256, 50256],
        [  220, 50256, 50256,  ..., 50256, 50256, 50256],
        ...,
        [ 5946, 16328,    82,  ..., 50256, 50256, 50256],
        [  220, 50256, 50256,  ..., 50256, 50256, 50256],
        [  796,   796,   796,  ..., 50256, 50256, 50256]], device='cuda:0')


UnboundLocalError: cannot access local variable 'unbindded_matricies' where it is not associated with a value